In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!rm -rf GazeGaussian
!git clone --recursive https://github.com/kram254/GazeGaussian.git
%cd GazeGaussian
!git submodule update --init --recursive

In [ ]:
!pip install --upgrade pip setuptools wheel ninja

In [ ]:
!pip install opencv-python h5py tqdm scipy scikit-image lpips kornia tensorboardX einops trimesh plyfile

In [ ]:
!pip uninstall -y diff-gaussian-rasterization simple-knn kaolin
!pip cache purge
!rm -rf /usr/local/lib/python3.*/dist-packages/diff_gaussian_rasterization*
!rm -rf /usr/local/lib/python3.*/dist-packages/simple_knn*
!rm -rf /usr/local/lib/python3.*/dist-packages/kaolin*
!rm -rf /content/GazeGaussian/submodules/diff-gaussian-rasterization/build
!rm -rf /content/GazeGaussian/submodules/simple-knn/build

In [ ]:
!pip install --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%cd submodules/diff-gaussian-rasterization
!python setup.py install
%cd ../..

In [ ]:
%cd submodules/simple-knn
!python setup.py install
%cd ../..

In [ ]:
%cd /content
!git clone --recursive https://github.com/NVIDIAGameWorks/kaolin
%cd kaolin
!python setup.py install
%cd /content/GazeGaussian

In [ ]:
import json
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/GazeGaussian_data/ETH-XGaze_test/ETH-XGaze_test")
h5_files = sorted([f.name for f in data_dir.glob("*.h5")])

print(f"Found {len(h5_files)} HDF5 files")
print(f"First 5 files: {h5_files[:5]}")

if not h5_files:
    print("\n❌ No .h5 files found! Please check your data path.")
else:
    train_split = int(len(h5_files) * 0.8)
    train_files = h5_files[:train_split]
    val_files = h5_files[train_split:]

    custom_config = {
        "train": train_files,
        "val": val_files,
        "val_gaze": val_files,
        "test": [],
        "test_specific": []
    }

    config_path = "/content/GazeGaussian/configs/dataset/eth_xgaze/train_test_split.json"
    with open(config_path, 'w') as f:
        json.dump(custom_config, f, indent=2)

    print(f"\n✓ Updated config at: {config_path}")
    print(f"  - Training files: {len(train_files)}")
    print(f"  - Validation files: {len(val_files)}")

In [ ]:
!cd /content/GazeGaussian && python train_meshhead.py \
    --img_dir /content/drive/MyDrive/GazeGaussian_data/ETH-XGaze_test/ETH-XGaze_test \
    --num_epochs 50 \
    --early_stopping \
    --patience 5